In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model

In [2]:
data = pd.read_csv('Airfoil.csv')

In [3]:
data.head()

,Frequency,Angle of Attack,Chord length,Free-stream velocity,Suction side displacement thickness,Scaled sound pressure level
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


In [4]:
data.columns

Index(['Frequency', 'Angle of Attack', 'Chord length', 'Free-stream velocity',
       'Suction side displacement thickness', 'Scaled sound pressure level'],
      dtype='object')

In [5]:
X = data.iloc[:,:-1]
y = np.array(data['Scaled sound pressure level'])

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=101)

In [8]:
import statsmodels.api as sm

C:\Users\karth\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [9]:
Xc = sm.add_constant(X_train)
lm_stats = sm.OLS(y_train,Xc)
stats_model = lm_stats.fit()
stats_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     235.5
Date:                Sun, 27 May 2018   Prob (F-statistic):          6.20e-172
Time:                        15:28:04   Log-Likelihood:                -3380.4
No. Observations:                1127   AIC:                             6773.
Df Residuals:                    1121   BIC:                             6803.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=======================================================================================================
                                          coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                 132.8266      0.638    208.070      0.000     131.574     134.079
Frequency                              -0.0012   4.68e-05    -26.276      0.000      -0.001      -0.001
Angle of Attack                        -0.4004      0.045     -8.802      0.000      -0.490      -0.311
Chord length                          -36.1022      1.931    -18.694      0.000     -39.891     -32.313
Free-stream velocity                    0.0957      0.010      9.989      0.000       0.077       0.115
Suction side displacement thickness  -152.7897     17.017     -8.979      0.000    -186.178    -119.401
==============================================================================
Omnibus:                        5.986   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.050   Jarque-Bera (JB):                7.576
Skew:                          -0.022   Prob(JB):                       0.0226
Kurtosis:                       3.399   Cond. No.                     5.25e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.25e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [10]:
X_test = sm.add_constant(X_test)
predictions_stats = stats_model.predict(X_test)

In [11]:
from sklearn.metrics import mean_squared_error

In [12]:
print(mean_squared_error(predictions_stats,y_test))

21.4884512023


In [13]:
#Gradient Descent

In [14]:
import random
import numpy as np

def random_w(p):
    return np.array([np.random.normal() for i in range(p)])

def hypothesis(X,w):
    return np.dot(X,w)

def loss(X,w,y):
    return hypothesis(X,w) - y

def squared_loss(X,w,y):
    return loss(X,w,y) ** 2

def gradients(X,w,y):
    gradients = list()
    n = float(len(y))
    for j in range(len(w)):
        gradients.append(np.sum(loss(X,w,y) * X[:,j]) / n)
    return gradients

def update(X,w,y,alpha=0.01):
    return [t - alpha*g for t,g in zip(w, gradients(X,w,y))]

def gradient_descent_runner(X, y, alpha = 0.01, eta = 10**-12, iterations = 20000):
    w = random_w(X.shape[1])
    print('Starting gradient descent at w = {0}, error = {1}'.format(w,np.sum(squared_loss(X,w,y))))
    for k in range(iterations):
        SSL = np.sum(squared_loss(X,w,y))
        new_w = update(X,w,y, alpha = alpha)
        new_SSL = np.sum(squared_loss(X,new_w,y))
        w = new_w
    return w,new_SSL       
    

In [15]:
from sklearn.preprocessing import StandardScaler

observations = len(X_train)
variables = data.columns[3:-1]
standardization = StandardScaler()
Xst = standardization.fit_transform(X_train)
Xst = np.column_stack((Xst,np.ones(observations)))

alpha = 0.001
w,new_SSL = gradient_descent_runner(Xst, y_train)
print('Final w = {0}, error = {1}'.format(w,new_SSL))

Starting gradient descent at w = [-0.89368392 -0.42502731  0.75155669 -0.46833669  2.43697975 -0.50599839], error = 17726964.85177095
Final w = [-4.1047912105259465, -2.3670499027517162, -3.3133227765677464, 1.4851234753821754, -2.0745145625783357, 124.65813309671624], error = 26592.456792770034


In [16]:
betas = w[:-1] / standardization.std_
bias = w[-1] - np.sum((standardization.mean_ / standardization.std_) * w[:-1])

coeffs = np.insert(betas,0,bias,axis=0)
coeffs

C:\Users\karth\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function std_ is deprecated; Attribute ``std_`` will be removed in 0.19. Use ``scale_`` instead
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\karth\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function std_ is deprecated; Attribute ``std_`` will be removed in 0.19. Use ``scale_`` instead
  warnings.warn(msg, category=DeprecationWarning)


array([  1.32826574e+02,  -1.22975300e-03,  -4.00430154e-01,
        -3.61021708e+01,   9.57384420e-02,  -1.52789700e+02])

In [17]:
coeffs = coeffs.reshape(1,6)

In [20]:
predictions_gd = np.array([np.dot(coeffs,(X_test.iloc[j,:].values.reshape(6,1))) for j in range(len(X_test))]).reshape(376,)

In [21]:
print(mean_squared_error(predictions_gd,y_test))

21.4884512023
